# Benchmark runs

## Initialization

In [1]:
import os
import sys

_path = os.path.abspath(os.path.join(".."))
if _path not in sys.path:
    sys.path.append(_path)
_path = os.path.abspath(os.path.join("..", "amber"))
if _path not in sys.path:
    sys.path.append(_path)

In [2]:
import numba as nb
import numpy as np
import tensorflow as tf
import time

from numpy import ndarray

from amber.bernstein import BPoly
from amber.bernstein import strides

In [3]:
np.random.seed(42)

## Hadamard product

In [4]:
def hadamard(A, B):
    return A * B

In [5]:
hadamard_ufunc = nb.vectorize(
    ["float32(float32, float32)", "float64(float64, float64)"]
)(hadamard)

In [6]:
def hadamard_core(A, B, C):
    m, n = A.shape
    m, n = B.shape
    for i in range(m):
        for j in range(n):
            C[i, j] = A[i, j] * B[i, j]

In [7]:
gu_hadamard = nb.guvectorize(
    [
        "float32[:,:], float32[:,:], float32[:,:]",
        "float64[:,:], float64[:,:], float64[:,:]",
    ],
    "(m,n),(n,p)->(n,p)",
)(hadamard_core)

In [8]:
n = 4000
A = np.random.rand(n, n)
B = np.random.rand(n, n)

### `numba`

In [9]:
C = hadamard_ufunc(A, B)
C

array([[3.08931628e-01, 5.92255739e-01, 2.40748463e-01, ...,
        3.92699366e-01, 3.68227284e-02, 5.31154445e-02],
       [1.47020185e-01, 5.39980163e-01, 6.07112164e-01, ...,
        3.47558479e-02, 3.32175384e-02, 3.89707026e-02],
       [8.60111703e-02, 5.13795234e-01, 4.98261245e-02, ...,
        2.57315353e-01, 8.27579130e-01, 2.61096130e-01],
       ...,
       [5.00911523e-02, 3.75422716e-01, 3.15418605e-01, ...,
        6.34747216e-01, 1.51912747e-01, 3.16547552e-01],
       [9.60042318e-02, 2.42357853e-01, 3.89347743e-01, ...,
        4.17583191e-01, 6.69831148e-01, 2.88874463e-01],
       [7.31558307e-04, 1.53147687e-01, 2.37510226e-01, ...,
        2.06726753e-01, 1.00185025e-01, 1.17237888e-02]])

In [10]:
%timeit hadamard_ufunc(A, B)

20.4 ms ± 400 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
C = gu_hadamard(A, B)
C

array([[3.08931628e-01, 5.92255739e-01, 2.40748463e-01, ...,
        3.92699366e-01, 3.68227284e-02, 5.31154445e-02],
       [1.47020185e-01, 5.39980163e-01, 6.07112164e-01, ...,
        3.47558479e-02, 3.32175384e-02, 3.89707026e-02],
       [8.60111703e-02, 5.13795234e-01, 4.98261245e-02, ...,
        2.57315353e-01, 8.27579130e-01, 2.61096130e-01],
       ...,
       [5.00911523e-02, 3.75422716e-01, 3.15418605e-01, ...,
        6.34747216e-01, 1.51912747e-01, 3.16547552e-01],
       [9.60042318e-02, 2.42357853e-01, 3.89347743e-01, ...,
        4.17583191e-01, 6.69831148e-01, 2.88874463e-01],
       [7.31558307e-04, 1.53147687e-01, 2.37510226e-01, ...,
        2.06726753e-01, 1.00185025e-01, 1.17237888e-02]])

In [12]:
%timeit gu_hadamard(A, B)

21.6 ms ± 643 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### `numpy`

In [13]:
C = A * B
C

array([[3.08931628e-01, 5.92255739e-01, 2.40748463e-01, ...,
        3.92699366e-01, 3.68227284e-02, 5.31154445e-02],
       [1.47020185e-01, 5.39980163e-01, 6.07112164e-01, ...,
        3.47558479e-02, 3.32175384e-02, 3.89707026e-02],
       [8.60111703e-02, 5.13795234e-01, 4.98261245e-02, ...,
        2.57315353e-01, 8.27579130e-01, 2.61096130e-01],
       ...,
       [5.00911523e-02, 3.75422716e-01, 3.15418605e-01, ...,
        6.34747216e-01, 1.51912747e-01, 3.16547552e-01],
       [9.60042318e-02, 2.42357853e-01, 3.89347743e-01, ...,
        4.17583191e-01, 6.69831148e-01, 2.88874463e-01],
       [7.31558307e-04, 1.53147687e-01, 2.37510226e-01, ...,
        2.06726753e-01, 1.00185025e-01, 1.17237888e-02]])

In [14]:
%timeit A * B

20.8 ms ± 390 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Matrix multiplication

An example from <http://numba.pydata.org/numba-doc/0.12/tutorial_numpy_and_numba.html>. In their example, however, their call to `gu_matmul` yields a wrong result.

In [15]:
def matmul_core(A, B, C):
    m, n = A.shape
    n, p = B.shape
    for i in range(m):
        for j in range(p):
            C[i, j] = 0.0
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

In [16]:
gu_matmul = nb.guvectorize(
    [
        "float32[:,:], float32[:,:], float32[:,:]",
        "float64[:,:], float64[:,:], float64[:,:]",
    ],
    "(m,n),(n,p)->(n,p)",
)(matmul_core)

Note: `numba` will fail to compute a result in reasonable time for `n = 2000`.

In [17]:
n = 1000
A = np.random.rand(n, n)
B = np.random.rand(n, n)

### `numba`

In [18]:
C = gu_matmul(A, B)
C

array([[253.39525479, 255.31030731, 249.37606905, ..., 250.83508135,
        247.90246571, 255.34742274],
       [260.51711872, 255.941561  , 255.791102  , ..., 250.55099364,
        249.71548221, 252.02718514],
       [245.10202543, 244.84066934, 244.83574251, ..., 243.54013448,
        236.98363105, 242.553251  ],
       ...,
       [248.91040042, 251.93615079, 247.53951545, ..., 247.33217846,
        243.80524779, 247.60013621],
       [247.13525441, 250.26624687, 250.11499605, ..., 243.77905079,
        241.03535335, 246.94327919],
       [255.79789086, 255.41624538, 257.08821581, ..., 251.68467025,
        249.10636069, 250.41409044]])

In [19]:
%timeit gu_matmul(A, B)

1.55 s ± 95.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### `numpy`

In [20]:
C = A @ B
C

array([[253.39525479, 255.31030731, 249.37606905, ..., 250.83508135,
        247.90246571, 255.34742274],
       [260.51711872, 255.941561  , 255.791102  , ..., 250.55099364,
        249.71548221, 252.02718514],
       [245.10202543, 244.84066934, 244.83574251, ..., 243.54013448,
        236.98363105, 242.553251  ],
       ...,
       [248.91040042, 251.93615079, 247.53951545, ..., 247.33217846,
        243.80524779, 247.60013621],
       [247.13525441, 250.26624687, 250.11499605, ..., 243.77905079,
        241.03535335, 246.94327919],
       [255.79789086, 255.41624538, 257.08821581, ..., 251.68467025,
        249.10636069, 250.41409044]])

In [21]:
%timeit A @ B

31.3 ms ± 858 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Linear interpolation

Another example from <http://numba.pydata.org/numba-doc/0.12/tutorial_numpy_and_numba.html>.

In [22]:
def lerp(a, b, t):
    return a + t * (b - a)

In [23]:
lerp_ufunc = nb.vectorize(
    ["float32(float32, float32, float32)", "float64(float64, float64, float64)"]
)(lerp)

In [24]:
n = 1000000
a = np.random.rand(n)
b = np.random.rand(n)
t = np.random.rand(n)

### `numba`

In [25]:
result = lerp_ufunc(a, b, t)
result

array([0.3393685 , 0.81892999, 0.43599322, ..., 0.15591659, 0.58145359,
       0.64624195])

In [26]:
%timeit lerp_ufunc(a, b, t)

1.27 ms ± 131 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### `numpy`

In [27]:
result = lerp(a, b, t)
result

array([0.3393685 , 0.81892999, 0.43599322, ..., 0.15591659, 0.58145359,
       0.64624195])

In [28]:
%timeit lerp(a, b, t)

1.91 ms ± 103 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Bernstein polynomials

In [29]:
n = 5
d = np.full(n, n)
m = strides(d)

In [30]:
b_poly = BPoly(d)

In [31]:
@nb.guvectorize(
    [
        "int64[:], int64[:], float32[:,:], float32[:,:], float32[:]",
        "int64[:], int64[:], float64[:,:], float64[:,:], float64[:]",
    ],
    "(n),(_),(__,howmany),(n,howmany)->(howmany)",
    writable_args=(2,),
)
def gu_b_poly(d, m, b, x, y):
    """
    An n-variate Bernstein polynomial.

    :param d: The degrees of the n-variate Bernstein polynomial.
    :param m: The strides within the n-variate Bernstein batch.
    :param b: The Bernstein batch.
    :param x: The n-variate input.
    :return: The value of the Bernstein polynomial.
    """
    n = x.shape[0]
    h = x.shape[1]
    for i in range(n):
        for j in range(m[i - 1] - m[i], 0, -m[i]):
            for k in range(j):
                for l in range(h):
                    b[k, l] += (b[m[i] + k, l] - b[k, l]) * x[i, l]
    for l in range(h):
        y[l] = b[0, l]

In [32]:
howmany = 10000
b = np.random.rand(m[n], howmany)
x = np.random.rand(n, howmany)

### `numba`

In [33]:
result = gu_b_poly(d, m, b.copy(), x)
result

array([0.46874691, 0.51766059, 0.49633476, ..., 0.5011208 , 0.48511396,
       0.50891279])

In [34]:
%timeit gu_b_poly(d, m, b.copy(), x)

410 ms ± 3.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### `numpy`

In [35]:
result = b_poly.eval(b, x)
result

array([0.46874691, 0.51766059, 0.49633476, ..., 0.5011208 , 0.48511396,
       0.50891279])

In [36]:
%timeit b_poly.eval(b, x)

798 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### `tensorflow`

In [40]:
result = b_poly(b, x)
result.numpy()

array([0.46874691, 0.51766059, 0.49633476, ..., 0.5011208 , 0.48511396,
       0.50891279])

In [41]:
%timeit b_poly(b, x)

163 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Does `tensorflow` cache calculations?

In [42]:
%timeit b_poly(b, np.random.rand(n, howmany)) 

157 ms ± 773 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


No, it does not.